# Lab Assignment-5
# Name- Manit Kakkar
# Roll No.-17CSU101

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing
import os

In [2]:
data=pd.read_csv(r'C:\Users\manit kakkar\OneDrive\Desktop\abcnews-date-text.csv')

In [3]:
data_text=data[:5000][['headline_text']];

In [4]:
data_text['index']=data_text.index

In [5]:
documents=data_text

In [6]:
print(len(documents))

5000


In [7]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [8]:
documents.isnull().sum()

headline_text    0
index            0
dtype: int64

## Data Preprocessing

In [9]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

D:\ana2\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
corpus=[]
for i in range(0,len(documents)):
    text=re.sub('[^a-zA-z]',' ',documents['headline_text'][i])
    text=text.lower()
    text=text.split()
    text=[PorterStemmer().stem(word) for word in text if not word in stopwords.words('english')]
    text=' '.join(text)
    corpus.append(text)
    

In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\manit
[nltk_data]     kakkar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [13]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [15]:
for i in range(0,len(corpus)):
    documents['headline_text'][i]=corpus[i]

D:\ana2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
documents[["headline_text"]]

,headline_text
0,aba decid commun broadcast licenc
1,act fire wit must awar defam
2,g call infrastructur protect summit
3,air nz staff aust strike pay rise
4,air nz strike affect australian travel
...,...
4995,slater star blue day one
4996,soprano film delay contract disput
4997,souri outlin region road fund
4998,south east water license pay levi


In [17]:
processed_docs=documents['headline_text'].map(preprocess)

In [18]:
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                                    [awar, defam]
2                 [infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5                    [ambiti, olsson, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Bag of words on the dataset

In [19]:
import gensim

In [20]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [21]:
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>20:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 infrastructur
7 protect
8 summit
9 aust
10 rise
11 staff
12 strike
13 affect
14 australian
15 travel
16 ambiti
17 jump
18 olsson
19 tripl
20 antic


In [22]:
dictionary.filter_extremes(no_below=15,no_above=0.1,keep_n=1000)

In [23]:
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [24]:
bow_corpus[100]

[(125, 1)]

In [25]:
bow_corpus_100=bow_corpus[100]
for i in range(len(bow_corpus_100)):
    print("Word {} (\"{}\") appears {} time.".format(bow_corpus_100[i][0], 
                                                     dictionary[bow_corpus_100[i][0]], 
                                                     bow_corpus_100[i][1]))

Word 125 ("women") appears 1 time.


## TF-IDF on our document set

In [26]:
from gensim import corpora,models
tfidf=models.TfidfModel(bow_corpus)

In [27]:
corpus_tfidf=tfidf[bow_corpus]

## Running LDA using Bag of Words

In [28]:
lda_model=gensim.models.LdaMulticore(bow_corpus,
                                    num_topics=10,
                                    id2word=dictionary,
                                    passes=2)

In [29]:
for idx,topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.037*"warn" + 0.031*"govt" + 0.031*"iraq" + 0.029*"protest" + 0.019*"head" + 0.019*"missil" + 0.018*"destroy" + 0.015*"tell" + 0.015*"crash" + 0.015*"crean"


Topic: 1 
Words: 0.055*"charg" + 0.035*"world" + 0.034*"murder" + 0.030*"kill" + 0.020*"polic" + 0.019*"hous" + 0.018*"trial" + 0.017*"drug" + 0.017*"claim" + 0.015*"boost"


Topic: 2 
Words: 0.042*"plan" + 0.024*"green" + 0.023*"play" + 0.020*"govt" + 0.019*"reject" + 0.018*"commun" + 0.017*"claim" + 0.017*"match" + 0.016*"offer" + 0.016*"iraqi"


Topic: 3 
Words: 0.052*"council" + 0.033*"polic" + 0.024*"death" + 0.022*"begin" + 0.019*"attack" + 0.018*"plan" + 0.017*"korea" + 0.016*"river" + 0.016*"downer" + 0.015*"record"


Topic: 4 
Words: 0.025*"coast" + 0.024*"govt" + 0.023*"nation" + 0.019*"help" + 0.018*"gold" + 0.015*"draw" + 0.015*"clash" + 0.014*"ahead" + 0.014*"woman" + 0.014*"hear"


Topic: 5 
Words: 0.033*"rain" + 0.031*"govt" + 0.029*"iraq" + 0.028*"council" + 0.022*"miss" + 0.021*"accus" + 0.020*"

## Running LDA using TF-IDF

In [30]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [31]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.037*"govt" + 0.023*"chang" + 0.023*"minist" + 0.021*"rule" + 0.021*"public" + 0.020*"record" + 0.019*"hold" + 0.018*"centr" + 0.017*"india" + 0.016*"talk"


Topic: 1 Word: 0.034*"world" + 0.024*"deni" + 0.020*"concern" + 0.019*"investig" + 0.018*"aust" + 0.018*"hospit" + 0.016*"unit" + 0.014*"downer" + 0.014*"issu" + 0.013*"caus"


Topic: 2 Word: 0.037*"plan" + 0.033*"polic" + 0.030*"charg" + 0.025*"iraq" + 0.021*"death" + 0.019*"murder" + 0.017*"jail" + 0.016*"chief" + 0.015*"crash" + 0.015*"increa"


Topic: 3 Word: 0.026*"trial" + 0.026*"hous" + 0.026*"kill" + 0.025*"group" + 0.025*"turn" + 0.019*"appeal" + 0.018*"elect" + 0.018*"charg" + 0.017*"iraq" + 0.016*"test"


Topic: 4 Word: 0.037*"warn" + 0.023*"fund" + 0.023*"farmer" + 0.022*"drought" + 0.021*"cyclon" + 0.019*"resid" + 0.018*"australia" + 0.017*"titl" + 0.017*"teacher" + 0.016*"accus"


Topic: 5 Word: 0.042*"open" + 0.027*"govt" + 0.025*"injur" + 0.025*"dead" + 0.020*"nation" + 0.020*"hewitt" + 0.019*"miss"

## Performance evaluation by classifying sample document using LDA Bag of Words model

In [32]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [33]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.48732778429985046	 
Topic: 0.074*"polic" + 0.029*"lead" + 0.020*"titl" + 0.019*"want" + 0.017*"fund" + 0.017*"work" + 0.017*"rail" + 0.016*"probe" + 0.015*"test" + 0.015*"appeal"

Score: 0.3792971074581146	 
Topic: 0.018*"anti" + 0.017*"plan" + 0.017*"cost" + 0.017*"price" + 0.016*"home" + 0.015*"pakistan" + 0.015*"resolut" + 0.015*"claim" + 0.014*"meet" + 0.014*"bushfir"

Score: 0.016678547486662865	 
Topic: 0.049*"iraq" + 0.031*"continu" + 0.030*"polic" + 0.026*"court" + 0.022*"fight" + 0.020*"group" + 0.019*"rain" + 0.019*"health" + 0.016*"return" + 0.016*"year"

Score: 0.01667364314198494	 
Topic: 0.037*"warn" + 0.031*"govt" + 0.031*"iraq" + 0.029*"protest" + 0.019*"head" + 0.019*"missil" + 0.018*"destroy" + 0.015*"tell" + 0.015*"crash" + 0.015*"crean"

Score: 0.016673551872372627	 
Topic: 0.033*"rain" + 0.031*"govt" + 0.029*"iraq" + 0.028*"council" + 0.022*"miss" + 0.021*"accus" + 0.020*"break" + 0.018*"meet" + 0.018*"farmer" + 0.017*"concern"

Score: 0.01667349040508270

# Performance evaluation by classifying sample document using LDA TF-IDF Bag of Words model

In [34]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.47668322920799255	 
Topic: 0.026*"trial" + 0.026*"hous" + 0.026*"kill" + 0.025*"group" + 0.025*"turn" + 0.019*"appeal" + 0.018*"elect" + 0.018*"charg" + 0.017*"iraq" + 0.016*"test"

Score: 0.38993892073631287	 
Topic: 0.025*"iraqi" + 0.019*"play" + 0.019*"hear" + 0.018*"report" + 0.018*"year" + 0.016*"offer" + 0.015*"push" + 0.015*"deal" + 0.014*"pressur" + 0.014*"send"

Score: 0.01667601987719536	 
Topic: 0.032*"work" + 0.024*"seek" + 0.020*"cost" + 0.019*"race" + 0.018*"stay" + 0.017*"turkey" + 0.017*"polic" + 0.017*"semi" + 0.016*"green" + 0.016*"arrest"

Score: 0.016675597056746483	 
Topic: 0.037*"govt" + 0.023*"chang" + 0.023*"minist" + 0.021*"rule" + 0.021*"public" + 0.020*"record" + 0.019*"hold" + 0.018*"centr" + 0.017*"india" + 0.016*"talk"

Score: 0.016675561666488647	 
Topic: 0.037*"council" + 0.025*"iraq" + 0.022*"bushfir" + 0.021*"face" + 0.017*"home" + 0.017*"polic" + 0.016*"claim" + 0.015*"resolut" + 0.015*"rain" + 0.015*"second"

Score: 0.016672620549798012	 
T

# Testing on Unseen Document

In [35]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5499815344810486	 Topic: 0.033*"rain" + 0.031*"govt" + 0.029*"iraq" + 0.028*"council" + 0.022*"miss"
Score: 0.0500054694712162	 Topic: 0.018*"anti" + 0.017*"plan" + 0.017*"cost" + 0.017*"price" + 0.016*"home"
Score: 0.05000368878245354	 Topic: 0.052*"council" + 0.033*"polic" + 0.024*"death" + 0.022*"begin" + 0.019*"attack"
Score: 0.05000288784503937	 Topic: 0.025*"coast" + 0.024*"govt" + 0.023*"nation" + 0.019*"help" + 0.018*"gold"
Score: 0.05000215396285057	 Topic: 0.037*"warn" + 0.031*"govt" + 0.031*"iraq" + 0.029*"protest" + 0.019*"head"
Score: 0.05000163987278938	 Topic: 0.055*"charg" + 0.035*"world" + 0.034*"murder" + 0.030*"kill" + 0.020*"polic"
Score: 0.05000099167227745	 Topic: 0.049*"iraq" + 0.031*"continu" + 0.030*"polic" + 0.026*"court" + 0.022*"fight"
Score: 0.05000053718686104	 Topic: 0.042*"plan" + 0.024*"green" + 0.023*"play" + 0.020*"govt" + 0.019*"reject"
Score: 0.05000053718686104	 Topic: 0.039*"open" + 0.028*"probe" + 0.022*"warn" + 0.021*"hope" + 0.021*"lau